In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('dataset/creditcard.csv')[:80_000]
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
X = df.drop(columns=['Time', 'Amount', 'Class']).values
y = df['Class'].values

In [5]:
print(f'Shapes of X, y = {X.shape}, {y.shape}')
print(f'#Fraud Cases = {y.sum()}')

Shapes of X, y = (80000, 28), (80000,)
#Fraud Cases = 196


In [16]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1_000)
model.fit(X,y)

LogisticRegression(max_iter=1000)

In [17]:
model.predict(X).sum()

151

# Class Weight

In [18]:
model = LogisticRegression(class_weight={0:1, 1:2}, max_iter=1_000)
model.fit(X,y).predict(X).sum()

171

In [19]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
    estimator=LogisticRegression(max_iter=1_000),
    param_grid={'class_weight': [{0:1, 1:v} for v in range(1, 4)]},
    cv=10, 
    n_jobs=-1
)

In [20]:
grid.fit(X,y)

GridSearchCV(cv=10, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 2},
                                          {0: 1, 1: 3}]})

In [21]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,3.655691,0.405497,0.004600,0.002858,"{0: 1, 1: 1}","{'class_weight': {0: 1, 1: 1}}",0.998500,0.997250,0.998125,0.998750,0.998375,0.9995,0.999750,0.998375,0.997500,0.998500,0.998463,0.000731,3
1,3.559392,0.406982,0.004969,0.005931,"{0: 1, 1: 2}","{'class_weight': {0: 1, 1: 2}}",0.998625,0.997250,0.998125,0.999625,0.998375,0.9995,0.999875,0.998500,0.998125,0.998750,0.998675,0.000761,2
2,3.162384,0.424557,0.002304,0.002998,"{0: 1, 1: 3}","{'class_weight': {0: 1, 1: 3}}",0.999250,0.997125,0.998125,0.999625,0.998500,0.9995,0.999875,0.998500,0.998125,0.999125,0.998775,0.000808,1


# Score Metric

In [23]:
lr = LogisticRegression()
lr.score

<bound method ClassifierMixin.score of LogisticRegression()>

In [24]:
??lr.score

Signature: lr.score(X, y, sample_weight=None)
Source:   
    def score(self, X, y, sample_weight=None):
        """
        Return the mean accuracy on the given test data and labels.

        In multi-label classification, this is the subset accuracy
        which is a harsh metric since you require for each sample that
        each label set be correctly predicted.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Test samples.

        y : array-like of shape (n_samples,) or (n_samples, n_outputs)
            True labels for `X`.

        sample_weight : array-like of shape (n_samples,), default=None
            Sample weights.

        Returns
        -------
        score : float
            Mean accuracy of ``self.predict(X)`` wrt. `y`.
        """
        from .metrics import accuracy_score
        return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
File:      c:\users\farooq khan\desktop\dev\sklearn-dat

In [25]:
from sklearn.metrics import precision_score, recall_score

In [29]:
print(f'Precision: {precision_score(y, grid.predict(X))}, Recall: {recall_score(y, grid.predict(X))}')

Precision: 0.8055555555555556, Recall: 0.7397959183673469


- Precision asks, give that I predicted fraud, how accurate am I?
- Recall asks, did I get all the fraud cases?

In [37]:
from sklearn.metrics import make_scorer
grid_metric = GridSearchCV(
    estimator=LogisticRegression(max_iter=1_000),
    param_grid={'class_weight': [{0:1, 1:v} for v in np.linspace(1,20,30)]},
    scoring={'precision': make_scorer(precision_score), 'recall': make_scorer(recall_score)},
    refit='precision', # optimize over precision
    cv=10, 
    return_train_score = True, 
    n_jobs=-1
)

In [38]:
grid_metric.fit(X, y)

GridSearchCV(cv=10, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'class_weight': [{0: 1, 1: 1.0},
                                          {0: 1, 1: 1.6551724137931034},
                                          {0: 1, 1: 2.310344827586207},
                                          {0: 1, 1: 2.9655172413793105},
                                          {0: 1, 1: 3.6206896551724137},
                                          {0: 1, 1: 4.275862068965517},
                                          {0: 1, 1: 4.931034482758621},
                                          {0: 1, 1: 5.586206896551724},
                                          {0: 1, 1: 6.241379310344827},
                                          {0: 1, 1: 6.896551724137931},...
                                          {0: 1, 1: 14.758620689655173},
                                          {0: 1, 1: 15.413793103448276},
                                          {0: 1, 1: 16.06896551724

In [42]:
pd.DataFrame(grid_metric.cv_results_)#.to_csv('results/grid_search_cv_linear_regression_precision_recall.csv', index=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,params,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,...,split2_train_recall,split3_train_recall,split4_train_recall,split5_train_recall,split6_train_recall,split7_train_recall,split8_train_recall,split9_train_recall,mean_train_recall,std_train_recall
0,3.658586,0.441277,0.016630,0.002751,"{0: 1, 1: 1.0}","{'class_weight': {0: 1, 1: 1.0}}",1.000000,0.463415,0.583333,1.000000,...,0.627119,0.548023,0.573864,0.573864,0.562500,0.613636,0.636364,0.607955,0.612185,0.054733
1,3.372206,0.454490,0.018593,0.004198,"{0: 1, 1: 1.6551724137931034}","{'class_weight': {0: 1, 1: 1.6551724137931034}}",1.000000,0.463415,0.583333,1.000000,...,0.683616,0.627119,0.670455,0.647727,0.630682,0.687500,0.698864,0.687500,0.680239,0.050286
2,3.321204,0.375700,0.017427,0.004376,"{0: 1, 1: 2.310344827586207}","{'class_weight': {0: 1, 1: 2.310344827586207}}",1.000000,0.463415,0.583333,1.000000,...,0.740113,0.683616,0.710227,0.698864,0.687500,0.715909,0.744318,0.727273,0.724454,0.043881
3,3.436434,0.416475,0.018141,0.006871,"{0: 1, 1: 2.9655172413793105}","{'class_weight': {0: 1, 1: 2.9655172413793105}}",1.000000,0.452381,0.583333,1.000000,...,0.785311,0.706215,0.744318,0.732955,0.715909,0.755682,0.772727,0.738636,0.749978,0.039589
4,2.848136,0.525705,0.018755,0.004254,"{0: 1, 1: 3.6206896551724137}","{'class_weight': {0: 1, 1: 3.6206896551724137}}",1.000000,0.452381,0.583333,1.000000,...,0.824859,0.740113,0.755682,0.744318,0.727273,0.778409,0.784091,0.761364,0.771498,0.037959
5,3.029555,0.590561,0.016272,0.002330,"{0: 1, 1: 4.275862068965517}","{'class_weight': {0: 1, 1: 4.275862068965517}}",1.000000,0.452381,0.583333,1.000000,...,0.841808,0.768362,0.778409,0.789773,0.772727,0.789773,0.795455,0.772727,0.792485,0.029289
6,3.129444,0.288047,0.018873,0.005818,"{0: 1, 1: 4.931034482758621}","{'class_weight': {0: 1, 1: 4.931034482758621}}",1.000000,0.452381,0.583333,1.000000,...,0.847458,0.802260,0.789773,0.818182,0.801136,0.801136,0.818182,0.789773,0.812327,0.021063
7,2.972758,0.452708,0.018228,0.004968,"{0: 1, 1: 5.586206896551724}","{'class_weight': {0: 1, 1: 5.586206896551724}}",1.000000,0.452381,0.583333,1.000000,...,0.847458,0.813559,0.829545,0.829545,0.812500,0.806818,0.835227,0.818182,0.827080,0.017190
8,3.104741,0.400176,0.018332,0.004105,"{0: 1, 1: 6.241379310344827}","{'class_weight': {0: 1, 1: 6.241379310344827}}",0.944444,0.452381,0.583333,0.947368,...,0.847458,0.824859,0.846591,0.835227,0.823864,0.829545,0.857955,0.829545,0.838431,0.013974
9,3.149776,0.587266,0.018041,0.003273,"{0: 1, 1: 6.896551724137931}","{'class_weight': {0: 1, 1: 6.896551724137931}}",0.944444,0.452381,0.583333,0.947368,...,0.853107,0.830508,0.846591,0.835227,0.829545,0.835227,0.863636,0.846591,0.844665,0.012015


In [ ]:
plt.figure(figsize=(12,4))
df = pd.DataFrame(grid_metric.cv_results_)
for score in ['mean_test_recall', 'mean_test_precision']:
    plt.plot([_[1] for _ in df['param_class_weight']], df[score],label=score)
plt.legend()